In [1]:
from dotenv import load_dotenv
import os 
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import hopsworks
import re 
import modal 


In [2]:
load_dotenv()

api_key = os.environ.get('stocks_api') # Replace this with your actual API key
ts = TimeSeries(key=api_key, output_format='pandas')

# Fetch daily adjusted stock prices; adjust the symbol as needed
data, meta_data = ts.get_daily(symbol='TSLA', outputsize='full')

print(data.head())

            1. open  2. high    3. low  4. close    5. volume
date                                                         
2024-04-30   186.98   190.95  182.8401    183.28  127031787.0
2024-04-29   188.42   198.87  184.5400    194.05  243869678.0
2024-04-26   168.85   172.12  166.3700    168.29  109815725.0
2024-04-25   158.96   170.88  158.3600    170.18  126427521.0
2024-04-24   162.84   167.97  157.5100    162.13  181178020.0


In [ ]:
# Define your file path and name
file_path = '/Users/manos/Documents/BDS/MLops_mod/TSLA_stock_price.csv'  # Customize the path and filename

# Save the DataFrame to CSV
data.to_csv(file_path)

print(f"Data saved to {file_path}")